## Atuação de redes neurais na base de dados do ENEM.

##### Alunos:

-   Gabriel Fonseca (2111066)
-   Yasmim Santos (2116925)
-   Alejandro Elias (2111189)
-   Pedro Lucas (2111131)

Base de dados escolhida - Exame Nacional do Ensino Médio (Enem): https://basedosdados.org/dataset/3e9c8804-c31c-4f48-9a45-d67f1c21a859


In [51]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split, GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical

In [52]:
df_enem = pd.read_csv("../data/out/enem-dados-tratados-ml.csv")

tts_r_state = 73
mlp_r_state = 83

X = df_enem.drop("label", axis=1).to_numpy()
y = df_enem["label"]

num_classes = len(y.unique())

y = y.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=tts_r_state
)

n_components = X.shape[1]

y_train_keras = to_categorical(y_train, num_classes=num_classes)
y_test_keras = to_categorical(y_test, num_classes=num_classes)

df_enem

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,pc11,pc12,pc13,label
0,0.080772,-0.762073,0.785344,-0.199764,-0.054483,0.049965,0.085057,-0.540182,0.122058,-0.009384,-0.204944,0.117868,0.051204,0
1,-0.393375,-0.548347,0.338718,0.221928,0.016524,-0.173460,-0.307259,-0.246581,-0.040625,-0.236048,0.013179,0.149297,-0.062338,3
2,0.776151,0.015068,-0.138433,-0.401180,-0.118839,0.141531,0.238757,-0.504716,0.119788,-0.021916,-0.216930,0.090747,0.061515,2
3,-0.298045,-0.212122,-0.284790,0.008414,-0.069669,-0.025346,0.005120,0.034136,-0.003425,0.037167,0.015733,-0.047331,-0.001086,4
4,-0.112868,-0.191347,-0.089860,0.090623,-0.013232,-0.043385,0.032071,0.104206,0.030562,0.156052,0.022517,-0.053376,0.016360,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176574,0.690036,0.275459,-0.179739,0.192360,0.070063,-0.119760,-0.097103,-0.064441,0.026953,-0.002614,0.016944,0.109682,-0.017169,2
176575,-0.864843,0.316883,-0.248853,-0.297550,-0.790818,-0.036361,-0.041141,-0.010166,0.254156,-0.300570,-0.073253,-0.256313,-0.678971,1
176576,-0.690970,-0.464787,-0.921498,-0.172432,-0.288687,-0.029190,-0.171915,-0.371058,-0.857355,0.223508,-0.043842,0.255755,0.332502,4
176577,-0.095767,-0.509823,0.567619,0.561381,0.035622,-0.666075,-0.746409,-0.182901,0.341742,0.691724,-0.216121,-0.045788,0.065079,3


1. ~~Diminuir o número de epocas~~
2. ~~Variar o batch size~~
3. ~~Testar outro modelo sem ser de deep learning~~
4. ~~Testar a variação das camadas ocultas utilizando tensorflow e colocando a função de ativação softmax na camada de saida~~


In [53]:
def class_grid_search(
    model: BaseEstimator,
    param_grid: dict,
    X_train: np.ndarray,
    y_train: np.ndarray,
    n_components: int,
):
    model_name = model.__class__.__name__.lower()

    grid_search = GridSearchCV(model, param_grid, n_jobs=-1, cv=5, scoring="accuracy")
    grid_search.fit(X_train, y_train)

    resultado_gs = pd.DataFrame(grid_search.cv_results_)
    resultado_gs = resultado_gs.sort_values(by="rank_test_score")

    resultado_gs.to_csv(
        f"../data/out/resultado_gridsearch_{model_name}_{n_components}.csv", index=False
    )

    resultado_gs = resultado_gs[
        ["params", "mean_test_score", "std_test_score", "rank_test_score"]
    ]

    return resultado_gs

In [54]:
model = Sequential(
    [
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation="relu"),
        Dense(32, activation="relu"),
        Dense(16, activation="relu"),
        Dense(5, activation="softmax"),
    ]
)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    X_train, y_train_keras, epochs=10, batch_size=32, validation_split=0.2
)
test_loss, test_accuracy = model.evaluate(X_test, y_test_keras)

print(f"Test accuracy: {test_accuracy}")

Epoch 1/10


2208/2208 ━━━━━━━━━━━━━━━━━━━━ 3s 863us/step - accuracy: 0.9376 - loss: 0.2565 - val_accuracy: 0.9972 - val_loss: 0.0077
Epoch 2/10
2208/2208 ━━━━━━━━━━━━━━━━━━━━ 2s 835us/step - accuracy: 0.9972 - loss: 0.0062 - val_accuracy: 0.9961 - val_loss: 0.0104
Epoch 3/10
2208/2208 ━━━━━━━━━━━━━━━━━━━━ 2s 789us/step - accuracy: 0.9982 - loss: 0.0047 - val_accuracy: 0.9977 - val_loss: 0.0047
Epoch 4/10
2208/2208 ━━━━━━━━━━━━━━━━━━━━ 2s 798us/step - accuracy: 0.9982 - loss: 0.0040 - val_accuracy: 0.9976 - val_loss: 0.0057
Epoch 5/10
2208/2208 ━━━━━━━━━━━━━━━━━━━━ 2s 814us/step - accuracy: 0.9980 - loss: 0.0049 - val_accuracy: 0.9978 - val_loss: 0.0047
Epoch 6/10
2208/2208 ━━━━━━━━━━━━━━━━━━━━ 2s 809us/step - accuracy: 0.9988 - loss: 0.0032 - val_accuracy: 0.9978 - val_loss: 0.0058
Epoch 7/10
2208/2208 ━━━━━━━━━━━━━━━━━━━━ 2s 799us/step - accuracy: 0.9988 - loss: 0.0033 - val_accuracy: 0.9976 - val_loss: 0.0057
Epoch 8/10
2208/2208 ━━━━━━━━━━━━━━━━━━━━ 2s 860us/step - accuracy: 0.9985 - loss: 0.00

In [55]:
perceptron = Perceptron(random_state=mlp_r_state)

param_grid = {
    "penalty": [None, "l2", "l1", "elasticnet"],
    "alpha": [1e-4, 1e-3, 1e-2, 1e-1],
    "fit_intercept": [True, False],
    "max_iter": [1000, 2000, 3000],
    "tol": [1e-3, 1e-4, 1e-5],
    "shuffle": [True, False],
    "eta0": [1.0, 0.1, 0.01],
    "early_stopping": [True, False],
    "validation_fraction": [0.1, 0.2, 0.3],
}

class_grid_search(perceptron, param_grid, X_train, y_train, n_components)

,params,mean_test_score,std_test_score,rank_test_score
1296,"{'alpha': 0.0001, 'early_stopping': False, 'et...",0.997021,0.000380,1
4325,"{'alpha': 0.001, 'early_stopping': False, 'eta...",0.997021,0.000380,1
6915,"{'alpha': 0.01, 'early_stopping': False, 'eta0...",0.997021,0.000380,1
4395,"{'alpha': 0.001, 'early_stopping': False, 'eta...",0.997021,0.000380,1
4396,"{'alpha': 0.001, 'early_stopping': False, 'eta...",0.997021,0.000380,1
...,...,...,...,...
9188,"{'alpha': 0.1, 'early_stopping': False, 'eta0'...",0.175140,0.086448,10360
9115,"{'alpha': 0.1, 'early_stopping': False, 'eta0'...",0.175140,0.086448,10360
9116,"{'alpha': 0.1, 'early_stopping': False, 'eta0'...",0.175140,0.086448,10360
9258,"{'alpha': 0.1, 'early_stopping': False, 'eta0'...",0.175140,0.086448,10360


In [56]:
mlp = MLPClassifier(random_state=mlp_r_state)

param_grid = {
    "hidden_layer_sizes": [(50,), (100,), (50, 50)],
    "activation": ["tanh", "relu"],
    "solver": ["adam"],
    "alpha": [1e-4, 1e-3],
    "learning_rate": ["constant", "adaptive"],
    "batch_size": [16, 32, 64],
    "max_iter": [30, 60, 90],
}

class_grid_search(mlp, param_grid, X_train, y_train, n_components)

,params,mean_test_score,std_test_score,rank_test_score
42,"{'activation': 'tanh', 'alpha': 0.0001, 'batch...",0.999298,0.000198,1
45,"{'activation': 'tanh', 'alpha': 0.0001, 'batch...",0.999298,0.000198,1
147,"{'activation': 'relu', 'alpha': 0.0001, 'batch...",0.999241,0.000337,3
144,"{'activation': 'relu', 'alpha': 0.0001, 'batch...",0.999241,0.000337,3
150,"{'activation': 'relu', 'alpha': 0.0001, 'batch...",0.999241,0.000185,5
...,...,...,...,...
65,"{'activation': 'tanh', 'alpha': 0.001, 'batch_...",0.998403,0.000241,211
64,"{'activation': 'tanh', 'alpha': 0.001, 'batch_...",0.998403,0.000241,211
62,"{'activation': 'tanh', 'alpha': 0.001, 'batch_...",0.998403,0.000241,211
63,"{'activation': 'tanh', 'alpha': 0.001, 'batch_...",0.998380,0.000223,215


In [59]:
perceptron_params = {
    "alpha": 0.0001,
    "early_stopping": False,
    "eta0": 0.1,
    "fit_intercept": False,
    "max_iter": 3000,
    "penalty": "l1",
    "shuffle": True,
    "tol": 1e-05,
    "validation_fraction": 0.3,
}

perceptron = Perceptron(**perceptron_params)

perceptron.fit(X_train, y_train)
test_accuracy = perceptron.score(X_test, y_test)

test_accuracy

0.9016309887869521